In [1]:
#Installing required packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions
import numpy as np

In [ ]:
#Reading the initial downloaded list of properties in the same neighborhood as the subject property
list = pd.read_csv('/Users/sonali/Documents/PersonalProjects/neighborhood_appraisals/neighborhood_appraisals.csv')
list.head()

In [ ]:
#Selecting the 'link' column as it is the WCAD link from where we can scrape more information for each property
link_list = list['Link']
len(link_list)

In [ ]:
#xpaths for all the variables we need to scrape along with the blank lists to store the scraped information
area_xpath = '//table[@class="improvementTable"]/tbody/tr[2]/td[5]'
propid_xpath = '//table[@id="dnn_ctr1460_View_tblPropertyHeaderInfo"]/tbody/tr[2]/td[1]'
imp_xpath1 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[2]'
imp_xpath2 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[3]'
imp_xpath3 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[4]'
imp_xpath4 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[5]'
imp_xpath5 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[6]'
imp_xpath6 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[7]'
imp_xpath7 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[8]'
imp_xpath8 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[9]'
imp_xpath9 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[10]'
imp_xpath10 = '//div[@id="dnn_ctr1460_View_divCamaInfo"]/ul/li/div/table/tbody/tr[11]'
expand_xpath = '//i[@class="fa fa-plus-square-o"]'
area = []
propid = []
imp1 = []
imp2 = []
imp3 = []
imp4 = []
imp5 = []
imp6 = []
imp7 = []
imp8 = []
imp9 = []
imp10 = []

In [ ]:
#Scraping house size (in square feet) and property ID for all the properties in the above list
driver = webdriver.Firefox(executable_path=r'/Users/sonali/Documents/geckodriver')
for link in link_list:
    driver.get(link)
    try:
        area_info = driver.find_element(By.XPATH, area_xpath)
        area.append(area_info.text)
    except NoSuchElementException:
        area.append('-')
    try:
        propid_info = driver.find_element(By.XPATH, propid_xpath)
        propid.append(propid_info.text)
    except NoSuchElementException:
        propid.append('-')

In [ ]:
#Creating a dataframe with scraped information
scrape = pd.DataFrame({'Property_ID':propid, 'area':area})
scrape.head()

In [ ]:
#Merging the scraped data with the initial list of all properties on "Property_ID"
scraped_data = pd.merge(list, scrape,on='Property_ID', how='left')
len(scraped_data)

In [ ]:
#Writing 'scraped_data' datafile (to avoid losing it later)
scraped_data.to_csv('/Users/sonali/Documents/PersonalProjects/scraped_data.csv', index=False)

In [ ]:
#Reading 'scraped_data' datafile
scraped_data = pd.read_csv('/Users/sonali/Documents/PersonalProjects/neighborhood_appraisals/Github/scraped_data.csv')
scraped_data.head()

In [ ]:
#Defining function to extract only numeric part from the 'area' column (2296 from 2,296 Sq. Ft)
import re
def extract_area(text):
    match = re.search(r'(\d+,\d+)', text)
    if match:
        return match.group()
    else:
        return ''

In [ ]:
#Applying above defined function to extract numeric part from the 'area' column
scraped_data['area_1'] = scraped_data['area'].apply(extract_area)
scraped_data.info()

In [ ]:
#Converting 'area' to numeric from object data type and saving it in a new variable 'area_1'
scraped_data['area_1'] = pd.to_numeric(scraped_data['area_1'].apply(lambda x: x.replace(',','')))

In [ ]:
#Filtering properties within 100 square feet of our own property size (which is 2698)
filt_scraped_data = scraped_data[(scraped_data['area_1']>2597) & (scraped_data['area_1']<2799)]
filt_scraped_data.info()

In [ ]:
#Writing 'filt_scraped_data' datafile (to avoid losing it later)
filt_scraped_data.to_csv('/Users/sonali/Documents/PersonalProjects/data/filt_scraped_data.csv', index=False)

In [ ]:
#Scraping improvements for filt_scraped_data in 3 parts (:40, 40:80 and 80:123)
link_list1 = filt_scraped_data['Link']
link_list1 = link_list1[:40]
len(link_list)

In [ ]:
#Scraping information on improvements for all the properties in the above list 'filt_scraped_data'
for link in link_list1:
    driver = webdriver.Firefox(executable_path=r'/Users/sonali/Documents/geckodriver')
    driver.get(link)
    try:
        propid_info = driver.find_element(By.XPATH, propid_xpath)
        propid.append(propid_info.text)
    except NoSuchElementException:
        propid.append('-')
    try:
        element = driver.find_element(By.XPATH, expand_xpath)
        driver.execute_script("arguments[0].scrollIntoView();", element)
        element.click()
        imp1_info = driver.find_element(By.XPATH, imp_xpath1)
        imp1.append(imp1_info.text)
    except NoSuchElementException:
        imp1.append('-')
    try:
        imp2_info = driver.find_element(By.XPATH, imp_xpath2)
        imp2.append(imp2_info.text)
    except NoSuchElementException:
        imp2.append('-')
    try:
        imp3_info = driver.find_element(By.XPATH, imp_xpath3)
        imp3.append(imp3_info.text)
    except NoSuchElementException:
        imp3.append('-')
    try:
        imp4_info = driver.find_element(By.XPATH, imp_xpath4)
        imp4.append(imp4_info.text)
    except NoSuchElementException:
        imp4.append('-')
    try:
        imp5_info = driver.find_element(By.XPATH, imp_xpath5)
        imp5.append(imp5_info.text)
    except NoSuchElementException:
        imp5.append('-')
    try:
        imp6_info = driver.find_element(By.XPATH, imp_xpath6)
        imp6.append(imp6_info.text)
    except NoSuchElementException:
        imp6.append('-')
    try:
        imp7_info = driver.find_element(By.XPATH, imp_xpath7)
        imp7.append(imp7_info.text)
    except NoSuchElementException:
        imp7.append('-')
    try:
        imp8_info = driver.find_element(By.XPATH, imp_xpath8)
        imp8.append(imp8_info.text)
    except NoSuchElementException:
        imp8.append('-')
    try:
        imp9_info = driver.find_element(By.XPATH, imp_xpath9)
        imp9.append(imp9_info.text)
    except NoSuchElementException:
        imp9.append('-')
    try:
        imp10_info = driver.find_element(By.XPATH, imp_xpath10)
        imp10.append(imp10_info.text)
    except NoSuchElementException:
        imp10.append('-')

In [ ]:
#Creating a dataframe with scraped information on improvements
imp_list1 = pd.DataFrame({'PROPERTY ID':propid,'imp1':imp1,'imp2':imp2,'imp3':imp3,'imp4':imp4,
                        'imp5':imp5,'imp6':imp6,'imp7':imp7,'imp8':imp8,'imp9':imp9,'imp10':imp10})
imp_list1.head()

In [ ]:
#Writing 'imp_list1' datafile (to avoid losing it later)
imp_list1.to_csv('/Users/sonali/Documents/PersonalProjects/imp_list1.csv', index=False)

In [ ]:
#Similarly scraped and wrote datafiles for 'filt_scraped_data' in 3 parts; reading the 3 datafiles
imp_list1 = pd.read_csv('/Users/sonali/Documents/PersonalProjects/data/imp_list1.csv')
imp_list2 = pd.read_csv('/Users/sonali/Documents/PersonalProjects/data/imp_list2.csv')
imp_list3 = pd.read_csv('/Users/sonali/Documents/PersonalProjects/data/imp_list3.csv')

In [ ]:
#Concatenating above 3 datafiles into single file
imp_list = pd.concat([imp_list1,imp_list2,imp_list3])

In [ ]:
#Writing 'imp_list' datafile (to avoid losing it later)
imp_list.to_csv('/Users/sonali/Documents/PersonalProjects/data/imp_list.csv', index=False)

In [ ]:
#Merging the scraped data 'imp_list' with the filtered list of properties 'filt_scraped_data' on "Property_ID"
prop_list = pd.merge(filt_scraped_data, imp_list,on='PROPERTY ID', how='left')

In [ ]:
#Writing 'prop_list' datafile (to avoid losing it later)
prop_list.to_csv('/Users/sonali/Documents/PersonalProjects/neighborhood_appraisals/prop_list.csv', index=False)

In [ ]:
#Reading 'prop_list' datafile
prop_list = pd.read_csv('/Users/sonali/Documents/PersonalProjects/neighborhood_appraisals/prop_list.csv')
prop_list.info()

In [3]:
#Defining function to extract improvement name from the improvement columns ('Garage' from '3 Garage 1994 420 N/A Details')
import re
def extract_imp_name(text):
    match = re.search(r'\d{1,2}\s?(.*?)\s?(\d+|-)', text)
    if match:
        return match.group(1)
    else:
        return ''

In [4]:
#Defining function to extract improvement area from the improvement columns ('420' from '3 Garage 1994 420 N/A Details')
import re
def extract_imp_area(text):
    match = re.search(r'\d{1,2}\s?.*?\s?(\d+|-)\s?(\d+,?\d*)', text)
    if match:
        return match.group(2)
    else:
        return ''

In [16]:
#Creating lists to store improvement area for each improvement ('420' from '3 Garage 1994 420 N/A Details')
imp1 = prop_list['imp1'].apply(extract_imp_area)
imp2 = prop_list['imp2'].apply(extract_imp_area)
imp3 = prop_list['imp3'].apply(extract_imp_area)
imp4 = prop_list['imp4'].apply(extract_imp_area)
imp5 = prop_list['imp5'].apply(extract_imp_area)
imp6 = prop_list['imp6'].apply(extract_imp_area)
imp7 = prop_list['imp7'].apply(extract_imp_area)
imp8 = prop_list['imp8'].apply(extract_imp_area)
imp9 = prop_list['imp9'].apply(extract_imp_area)
imp10 = prop_list['imp10'].apply(extract_imp_area)

In [19]:
#Creatng column 'Fireplace' having number of fireplaces for each property
prop_list['Fireplace'] = np.where(prop_list['imp1'].apply(extract_imp_name)=='Fireplace',
                                  pd.to_numeric(imp1.apply(lambda x: x.replace(',',''))),0)
prop_list['Fireplace'] = np.where(prop_list['imp2'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp2.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp3'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp3.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp4'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp4.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp5'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp5.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp6'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp6.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp7'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp7.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp8'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp8.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp9'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp9.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])
prop_list['Fireplace'] = np.where(prop_list['imp10'].apply(extract_imp_name)=='Fireplace',
                                  prop_list['Fireplace']+pd.to_numeric(imp10.apply(lambda x: x.replace(',',''))),prop_list['Fireplace'])

In [20]:
#Creatng column 'Garage' having total area for garage for each property
prop_list['Garage'] = np.where(prop_list['imp1'].apply(extract_imp_name)=='Garage',
                                  pd.to_numeric(imp1.apply(lambda x: x.replace(',',''))),0)
prop_list['Garage'] = np.where(prop_list['imp2'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp2.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp3'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp3.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp4'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp4.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp5'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp5.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp6'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp6.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp7'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp7.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp8'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp8.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp9'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp9.apply(lambda x: x.replace(',',''))),prop_list['Garage'])
prop_list['Garage'] = np.where(prop_list['imp10'].apply(extract_imp_name)=='Garage',
                                  prop_list['Garage']+pd.to_numeric(imp10.apply(lambda x: x.replace(',',''))),prop_list['Garage'])

In [22]:
#Creatng column 'OpenPorch' having total area for Open Porch for each property
prop_list['OpenPorch'] = np.where(prop_list['imp1'].apply(extract_imp_name)=='Open Porch',
                                  pd.to_numeric(imp1.apply(lambda x: x.replace(',',''))),0)
prop_list['OpenPorch'] = np.where(prop_list['imp2'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp2.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp3'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp3.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp4'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp4.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp5'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp5.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp6'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp6.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp7'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp7.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp8'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp8.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp9'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp9.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])
prop_list['OpenPorch'] = np.where(prop_list['imp10'].apply(extract_imp_name)=='Open Porch',
                                  prop_list['OpenPorch']+pd.to_numeric(imp10.apply(lambda x: x.replace(',',''))),prop_list['OpenPorch'])

In [23]:
#Creatng column 'Deck' having total area for Deck for each property
prop_list['Deck'] = np.where(prop_list['imp1'].apply(extract_imp_name)=='Deck',
                                  pd.to_numeric(imp1.apply(lambda x: x.replace(',',''))),0)
prop_list['Deck'] = np.where(prop_list['imp2'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp2.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp3'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp3.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp4'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp4.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp5'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp5.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp6'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp6.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp7'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp7.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp8'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp8.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp9'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp9.apply(lambda x: x.replace(',',''))),prop_list['Deck'])
prop_list['Deck'] = np.where(prop_list['imp10'].apply(extract_imp_name)=='Deck',
                                  prop_list['Deck']+pd.to_numeric(imp10.apply(lambda x: x.replace(',',''))),prop_list['Deck'])

In [24]:
#Creatng column 'Patio' having total area for Patio for each property
prop_list['Patio'] = np.where(prop_list['imp1'].apply(extract_imp_name)=='Patio',
                                  pd.to_numeric(imp1.apply(lambda x: x.replace(',',''))),0)
prop_list['Patio'] = np.where(prop_list['imp2'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp2.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp3'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp3.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp4'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp4.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp5'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp5.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp6'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp6.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp7'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp7.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp8'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp8.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp9'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp9.apply(lambda x: x.replace(',',''))),prop_list['Patio'])
prop_list['Patio'] = np.where(prop_list['imp10'].apply(extract_imp_name)=='Patio',
                                  prop_list['Patio']+pd.to_numeric(imp10.apply(lambda x: x.replace(',',''))),prop_list['Patio'])

In [25]:
#Calculating adjusments for Fireplace, Garage, Open Porch, Deck and Patio
prop_list['Fireplace_adj'] = (1-prop_list['Fireplace'])*12354
prop_list['Garage_adj'] = (483-prop_list['Garage'])*19.47
prop_list['OpenPorch_adj'] = (64-prop_list['OpenPorch'])*31.81
prop_list['Deck_adj'] = (588-prop_list['Deck'])*47.35
prop_list['Patio_adj'] = (0-prop_list['Patio'])*34.75

In [27]:
#Creating new column for size adjustment
prop_list['size_adj'] = (2698-prop_list['area_1'])*75

In [ ]:
#Converting 'Value_2023' to numeric from object type
prop_list['Value_2023'] = pd.to_numeric(prop_list['Value_2023'].apply(lambda x: x.replace(',', '').replace('$', '')))

In [28]:
#Calculating appraisal value with all the adjustments as 'app_value' variable
prop_list['app_value'] = (prop_list['size_adj']+prop_list['Fireplace_adj']+prop_list['Garage_adj']+prop_list['OpenPorch_adj']+
prop_list['Deck_adj']+prop_list['Patio_adj']+prop_list['Value_2023'])

In [31]:
#Writing the datafile as 'final_list.csv'
prop_list.to_csv('/Users/sonali/Documents/PersonalProjects/neighborhood_appraisals/final_list.csv', index=False)